# CA2 : The Decoding Problem
The project's aim is to decode an encoded text. This encoding, maps each alphabetic character to another one. These letters has been replaced with eachother. As there are *26!* possible ways(we have 26 alphabetic character) to do so, the deconding cannot be done by just exploring all of them. As a result, we tend to find a way to decode the encoded text given as input data using **Genetic Algorithm**. 

## Basic Concepts
To solve this problem using **Genetic Algorithm**, some basic units should be defined first.<br>
 **1. Genes** <br>
In this problem, genes are a number between *0* to *26*, each indicating an alphabetic character. <br>
 **2. Chromosomes** <br>
Each chromosome, is a numpy array consisting of 26 Genes(each indicating a letter of alphabet), showing a possible answer for this problem. <br>
 **3. Population** <br>
Population is a group of chromosomes, which are possible solutions to this problem. This model aims to find new chromosomes based on combining chromosomes in its population, and make a new generation to find better answers.
<br>

## Input Data
Two *.txt* files are given as the input data of this problem. The first one is *global_test.txt* which is the source file of this problem. As it was said in the project, we need to make a dictionary using this file. The instructions to do so, has been explained in later parts. This file is a set of characters contining white spaces, alphabetic characters and other punctuations. <br>
The second file is *encoded_text.txt*, which is a text file including an incoded file using some key explained befire. The aim is to find the key and translate this file. This file contains the same characters as well.

## Libraries

`time` library is used to measure the time of this code. <br>
`string` library is then used to have ascii characters as a string to be processed.
`IPython.display` library is used to display the result in markdown.
`re` library is solving one of the key problems that is extracting words from the source data available. <br>
As we know, `numpy` is one of the fastest libraries available nowadays, so it has been chosen to process data with it. <br>


In [2]:
import re 
import time
import numpy as np  
from IPython.display import Markdown
from string import ascii_lowercase, ascii_uppercase

## The Decoder
To solve this problem, a solver class called **Decoder** is being defined, in which the original text we need to decode, its words, and the dictionary that is being extracted from *global_text.txt* given as our source of data is being kept.
<br><br>
The functions of this decoder are being explained below:
### 1. `__init__()` (constructor)
In this function, which is being called after creating an instance of *Decoder* class, all the needed information is being initialized and after that, these data need some operations to be usable. The most important part of organizing input data is to extract needed ones from the hole package. To do so, we tend to follow the instructions explained below:
##### - Extracting data from global file given as source
To use this data as the source dictionary, we need to extract its words first. To do so, `re` library is used. **findall()** function is being used to find all the strings just including alphabet, split them into words and ignoring other characters. The output of this function, is a list of extracted strings. To have a dictionary without any duplicated word, we convert all of these data from a *list* to a *set*. Now we have a set of words as a dictionary to compare decoded words with it.

### 2. `create_population()`
Based on the definition of population explained before, this function tends to create one based on randomness. It means that the function is making random chromosomes to create a population with the length given to it. For making chromosomes with unique Genes, it uses `random.choice()` function from `numpy` library.

### 3. `crossover()`
As it was mentioned before, *Decoder* needs to generate new chromosomes and replace those good ones with some old chromosomes in the population in order to find better possible answers. To do so, this function is being used. Here, it tends to mix up two possible answers from population and create new chromosomes which may be more close to the final answer. <br>
To implement this method, first a random index should be found(partition) to mix two chromosomes(parents) using that. Then, child1 would inherit its first part from the first parent(from first element to the partition index) and its second part(from the partition index to the last element) from the second parent, and vice versa. <br>
It should be noted that for each chromosome, there must not be any duplicated Gene, and to prevent this from happening, this code will replace these genes with some random gene that has not been in the chromosome yet.<br>
Another important part of this function is about **mutation**. It has been implemented as a function called `apply_mutation()` and it will be explained in the next part.

### 4. `apply_mutation()`
As we explained in the previous part, crossover function will provide the decoder with two new chromosomes extracted from the ones it had before. The problem is that sometimes the answer id going to some specific place which is not the final answer but with the fixed chromosomes in the population, the *Decoder* is not allowed to make a new one. So the solver will have a local solution and not the optimal one.To prevent this from happening, this function is being used. It will happen with the probablity fixed for the function(*mutation rate*), in which it swap the place of two *Genes* in a chromose randomly, so there would be a possibility to find a new answer which may be a better one. <br>
To implement this, we use the `random.choice()` function from `numpy` library again to have a random number wich can be 0 or 1 with the probablity of **mutation rate**. Then if the function is allowed to swap two *Genes*, it would randomly choose two of them in the chromosome and swap them.

### 5. `create_dict_ascii()`
This function tends to map the chromosomes given to it to the alphabets in order to decode the encoded text with these keys. As this decoder is keeping numbers instead of each letter, it first needs to map each number to its letter. It is known that 0 is indicating *'a'* or *'A'* character and 25 is indicating *'z'* or *'Z'* character. This function first maps each chromosome with a string including these letters in order. Then it needs to map them to the real alphabetics. To do so, a function called `maketrans()` from `string` library is being used to make a dictionary which links each alphabet to its coded alphabet.

### 6. `fit_fcn()` (Fitting Function)
This function is a measure for each chromosome that specifies how good it is.For having a fair measurement, it has been chosen to count the length of each word that has a meaning in the phrase(after decoding the encoded text using this chromosome). <br>
To do so, after decoding the encoded text, its words are being searched in the dictionary created from *global_text.txt* and the length of the meaningful ones will be counted. The final output of *fitting function* is the some of these lengths.

$$output = \sum\limits_{word \in decoded text \ \& \ dictionary } length(word)$$

### 7. `evaluate()`
This fuction tends to return an array consisting of all fitting values of the Decoder's population(for each chromosome).

### 8. `create_next_generation()`
The aim of this function is to mix up current population and create a new group of chromosomes as the next generation. As we know, this generation must have some better chromosomes which are closer to the final solution. To do so, it needs to choose better chromosomes to mix up together using `crossover()` function. For finding these chromosomes, the function is using `random.choice()` function from `numpy` library, in which the probablity of choosing each chromosome is :
$$ P = \frac{fitness\_of\_selected\_chromosomes}{\sum\limits_{chromosome \in all chromosomes}fitting\_fcn(chromosome)} $$
with this probablity, each time two chromosomes from population will be chosen and then two new chromosomes will be created. It should be noted that at first, fitness values of all the chromosomes had been calculated using `evaluate()` function.

### 9. `merge_generations()`
After creating a new generation, the population needs to be updated because chromosomes need converging to the optimal answer. To do so, this function is being used, in which it sorts all the chromosomes available(current population and new generation) based on their fitting value and then cuts off what is more than needed. More specific, in keeps the best chromosomes(the number of them are equal to the length of population). Finally, former popualtion will be replaced with this new population consisting of best chromosomes in two generations.

### 10. `find_max_point()`
This function returns the maximum amount of fitting value that can ever be reached(based on the policy of fitting function). It is being used to evaluate the best solution founded in each iteration later.

### 11. `decode()`
The last step is to implement the whole **Genetic Algorithm** using functions and concepts explained before.To do so, it needs to follow these steps: <br>
1. It has a while loop to iterate as much as needed to find the final answer in its popultaion. <br>
2. In each iteration population will be updated first(using functions explaind before). <br>
3. After updating the population, using `evaluate()` function the population will be evaluated and its best solution will be found. <br>
4. By comparing the best founded answer with the fitness value of the optimal answer, we evaluate the possible answer and if it was optimal, the loop will be ended and the answer is the best chromosome in population. Otherwise, The loop will be started again.
<br>
At last the decoder will find the key of this problem and return the decoded text.

### 12. `show_all_results()`
This function shows all the details about decoding the encoded text.

In [3]:
class Decoder:
    def __init__(self, phrase, dict_string, num_of_population):
        self.dictionary = set(re.findall(r'[a-zA-Z]+', dict_string))
        self.phrase = phrase
        self.phrase_words = re.findall(r'[a-zA-Z]+', phrase)
        self.population = self.create_population(num_of_population)
        self.max_point = self.find_max_point()
        self.final_chromosome = None
        self.num_of_iterations = 0
        self.time = 0
    
    def create_population(self, sizeof_population):
        population = np.zeros((sizeof_population, 26), dtype=int)
        for x in range(sizeof_population):
            population[x] = np.random.choice(26, 26, replace=False)
        return population

    def crossover(self, first_parent, second_parent, mutation_rate):
        partition_index = np.random.randint(first_parent.size)            

        child1 = -1 * np.ones(first_parent.shape, dtype=int)
        child2 = -1 * np.ones(first_parent.shape, dtype=int)
        
        for i in range(partition_index):
            child1[i] = first_parent[i]
            child2[i] = second_parent[i]
        for i in range(partition_index+1, len(child1)):
            if not second_parent[i] in child1:
                child1[i] = second_parent[i]
            if not first_parent[i] in child2:
                child2[i] = first_parent[i]

        for index in range(len(child1)):
            if child1[index] != -1 and child2[index] != -1:
                continue
            for x in range(26):
                if not x in child1:
                    child1[index] = x
                if not x in child2:
                    child2[index] = x
                    break
        child1 = self.apply_mutation(mutation_rate, child1)
        child2 = self.apply_mutation(mutation_rate, child2)
                             
        return child1, child2
    
    def apply_mutation(self, mutation_rate, chromosome):
        if np.random.choice(2, 1, p=[1 - mutation_rate, mutation_rate]) == 1:
            mutation_index_1 = np.random.randint(len(chromosome))
            mutation_index_2 = np.random.randint(len(chromosome))
            chromosome[mutation_index_1], chromosome[mutation_index_2] = \
                    chromosome[mutation_index_2], chromosome[mutation_index_1]
        return chromosome
    
    def create_dict_ascii(self, chromosome):
        all_ascii =ascii_lowercase + ascii_uppercase
        lower_case = ""
        upper_case = ""
        for i in range(len(chromosome)):
            lower_case += chr(ord('a') + chromosome[i])
            upper_case += chr(ord('A') + chromosome[i])
        dict = str.maketrans(all_ascii, lower_case+upper_case)
        return dict
    
    def fit_fcn(self, child):
        ans = 0
        dict = self.create_dict_ascii(child)
        for w in self.phrase_words:
            if w.translate(dict) in self.dictionary:
                ans += len(w)
                
        return ans
    
    def evaluate(self):
        answer = np.zeros(self.population.shape[0], dtype=int)
        for i in range(len(answer)):
            answer[i] = self.fit_fcn(self.population[i])
        return answer
    
    def create_next_generation(self,mutation_rate):
        fitness = self.evaluate()
        next_generation = np.zeros(self.population.shape, dtype=int)

        for i in range(0, int(self.population.shape[0]/2)):
            parent1_index, parent2_index = np.random.choice(self.population.shape[0], 2, p=fitness / sum(fitness))
            next_generation[2*i], next_generation[2*i+1] = \
                self.crossover(self.population[parent1_index, :], self.population[parent2_index, :], mutation_rate)

        return next_generation
     
    def merge_generations(self, children_gen):
        total_population = np.concatenate((self.population, children_gen), axis=0)

        total_fitness = np.zeros(total_population.shape[0], dtype=int)
        for i in range(len(total_fitness)):
            total_fitness[i] = self.fit_fcn(total_population[i])

        sorted_indices = sorted(range(len(total_fitness)), key=lambda k: total_fitness[k], reverse=True)
        total_population = total_population[sorted_indices, :]
        
        return total_population[:self.population.shape[0], :]
    
    def find_max_point(self):
        answer = 0
        for i in range(len(self.phrase_words)):
            answer += len(self.phrase_words[i])
        return answer
    
    def decode(self, mutation_rate=0.7):
        start = time.time()
        while True:
            new_population = self.create_next_generation(mutation_rate)
            self.population = self.merge_generations(new_population)
            population_fitness = self.evaluate()
            max_fitness = np.argmax(population_fitness)
            if np.amax(population_fitness) == self.max_point:
                break
            self.num_of_iterations += 1
            
        max_fit = np.argmax(population_fitness)    
        self.time = time.time()-start
        self.final_chromosome = self.population[max_fit]
        dict = self.create_dict_ascii(self.final_chromosome)

        return self.phrase.translate(dict)
    
    def show_all_results(self):
        dict = self.create_dict_ascii(self.final_chromosome)
        
        print("Total time:", self.time)
        print("Total number of iterations:", self.num_of_iterations)
        print("The founded key is shown below:")
        
        print("{", end="")
        key = ascii_lowercase.translate(dict)
        for i in range(len(self.final_chromosome)):
            if i != 0:
                print(", ", end="")
            print(ascii_lowercase[i], ":", key[i], end="")
        print("}")
        

## Results
The founded key is shown below in a table. This code will find the Key with average time of **55 Seconds** and **80 Iterations**. To show the results the `show_all_results()` can be called, but is has not been called due to the needs of the project. It can be easily called as `decoder.show_all_results()`
<br>
It should be noted that the output have been printed with IPython.display library. It can be easily print normally as well.
<br><br>

| Letter | A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z 
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| Key | O | R | S | F | W | M | B | T | I | Z | G | H | K | N | V | E | L | P | D | J | C | U | Y | Q | A | X |
<caption  style="text-align:center">The Final Key of Given Problem</caption>

In [4]:
dict_string = open("global_text.txt", "r").read()
phrase = open("encoded_text.txt", "r").read()
decoder = Decoder(phrase, dict_string, 500)
decoded_text = decoder.decode()
display(Markdown("### The decoded text is shown below:\n" + decoded_text))

### The decoded text is shown below:
This response originally fell into a bit bucket.  I'm reposting it
just so Bill doesn't think I'm ignoring him.

In article <C4w5pv.JxD@darkside.osrhe.uoknor.edu> bil@okcforum.osrhe.edu (Bill Conner) writes:
>Jim Perry (perry@dsinc.com) wrote:
>
>[Some stuff about Biblical morality, though Bill's quote of me had little
> to do with what he goes on to say]

Bill,

I'm sorry to have been busy lately and only just be getting around to
this.

Apparently you have some fundamental confusions about atheism; I think
many of these are well addressed in the famous FAQ.  Your generalisms
are then misplaced -- atheism needn't imply materialism, or the lack
of an absolute moral system.  However, I do tend to materialism and
don't believe in absolute morality, so I'll answer your questions.

>How then can an atheist judge value? 

An atheist judges value in the same way that a theist does: according
to a personal understanding of morality.  That I don't believe in an
absolute one doesn't mean that I don't have one.  I'm just explicit,
as in the line of postings you followed up, that when I express
judgment on a moral issue I am basing my judgment on my own code
rather than claiming that it is in some absolute sense good or bad.
My moral code is not particular different from that of others around
me, be they Christians, Muslims, or atheists.  So when I say that I
object to genocide, I'm not expressing anything particularly out of
line with what my society holds.

If your were to ask why I think morality exists and has the form it
does, my answer would be mechanistic to your taste -- that a moral
code is a prerequisite for a functioning society, and that humanity
probably evolved morality as we know it as part of the evolution of
our ability to exist in large societies, thereby achieving
considerable survival advantages.  You'd probably say that God just
made the rules.  Neither of us can convince the other, but we share a
common understanding about many moral issues.  You think you get it
from your religion, I think I get it (and you get it) from early
childhood teaching.

>That you don't like what God told people to do says nothing about God
>or God's commands, it says only that there was an electrical event in your
>nervous system that created an emotional state that your mind coupled
>with a pre-existing thought-set to form that reaction. 

I think you've been reading the wrong sort of comic books, but in
prying through the gobbledygook I basically agree with what you're
saying.  I do believe that my mental reactions to stimuli such as "God
commanded the genocide of the Canaanites" is mechanistic, but of
course I think that's true of you as well.  My reaction has little to
do with whether God exists or even with whether I think he does, but
if a god existed who commanded genocide, I could not consider him
good, which is supposedly an attribute of God.

>All of this being so, you have excluded
>yourself from any discussion of values, right, wrong, goood, evil,
>etc. and cannot participate. Your opinion about the Bible can have no
>weight whatsoever.

Hmm.  Yes, I think some heavy FAQ-reading would do you some good.  I
have as much place discussing values etc. as any other person.  In
fact, I can actually accomplish something in such a discussion, by
framing the questions in terms of reason: for instance, it is clear
that in an environment where neighboring tribes periodically attempt
to wipe each other out based on imagined divine commands, then the
quality of life will be generally poor, so a system that fosters
coexistence is superior, if quality of life is an agreed goal.  An
absolutist, on the other hand, can only thump those portions of a
Bible they happen to agree with, and say "this is good", even if the
act in question is unequivocally bad by the standards of everyone in
the discussion.  The attempt to define someone or a group of people as
"excluded from discussion", such that they "cannot participate", and
their opinions given "no weight whatsoever" is the lowest form or
reasoning (ad hominem/poisoning the well), and presumably the resort
of someone who can't rationally defend their own ideas of right,
wrong, and the Bible.
-- 
Jim Perry   perry@dsinc.com   Decision Support, Inc., Matthews NC
These are my opinions.  For a nominal fee, they can be yours.

-- 
Jim Perry   perry@dsinc.com   Decision Support, Inc., Matthews NC
These are my opinions.  For a nominal fee, they can be yours.


## Considerations and Explanations

### 1. Cleaning the input data and extract a dictionary
As it was explained before(in defining and explaining the constructor of the decoder), the words of the file *global_text.txt* had been extracted using `findall()` function from `re` library and changed into a list of words. In fact, this function will eliminate all the white spaces and other unnecessary elements in the encoded text. These marks will not change in the process of decoding and encoding. <br>
The aim of making such a dictionary is to have a reference for evaulating different chromosomes. Every word decoded using the chromosome is being compared with the words extracted from the dictionary.

### 2. Increasing number of chromosomes in population
To have an appropriate result, an adequate number of chromosomes is one of the basic needs, because an inadequate number of chromosomes will not have the variety needed to test possible ways and converge to the best one. <br>
On the other hand, a huge number of chromosomes in each generation, will take too much time for just processing the data. So, a reasonable number of population would be something in between. By examining the code, a total number of **500** chromosomes for each generation(number of chromosomes of the population) was the best choice for leading us to the final solution.

### 3. The reason for using mutations
Obviously, with having a fixed generation and just extracnting new generations from those fixed chromosomes, will give a direction to newly made chromosomes and as a result, the answer will converge to a answer which is not necessarily the optimal answer and just come to a local minimum/maximum. <br>
To prevent this from happening, using mutations is a vital fact. By changing some *Genes* randomly, the next generation will have some randomly changes which can find a better solution that was not in the general direction.

### 4. Crossover Vs. Mutation
There should be a reasonable trade-off between these two. As explained before, having just crossover will just randomly find the optimal solution. In most of the cases, this would just lead to a local minimum/maximum and not the best it can find. <br>
However, mutation just uses randomness which leads to a completely random population. This method may find the optimal answer accidentally. So this will not be the way to find optimal solution. <br>
All in all, there must be trade-off between these two to find an optimal solution as neither of these methods can find the needed result individually.

### 5. Possible solutions to prevent local optimums
Although *Genetic Algorithm* is using mutation with its crossover, it is still possible somtimes to stuck in local optimums and lose the right direction. To prevent this from happening, we had examined several ways from which some of them were proved truely effective.<br>
**1. Number of chromosomes** <br>
As explaied before, in order to address the problem of not having enough variety, it is necessary to choose an adequate number for the population length. So finding a number not too big or small, will help the decoder to find optimal solution in a reasonable time. In this problem, the total number of **500** chromosomes have been chosen for the number of population.<br>
**2. Mutation rate** <br>
One way to prevent answers from converging is to increase the mutation rate. <br>
It is known that mutation brings randomness so increasing this parameter can prevent the decoder from getting stuck in local optimums. To address this problem, the probablity of **0.7** for each chromosome(and not each *Gene*) have been chosen. <br>
**3. Using probablity to choose parents** <br>
As it was mentioned in the explanation of `creat_next_generation()` function, in this part of the decoder chromosomes are being chosen based on a probablity. If they have a higher fitting value, they are more likely to be chosen as the parent, but it is not definite. In other words, every chromosome have the chance to be chosen as the parent chromosome. This way will lead to use a little bit of randomness in choosing parent chromosomes and therefore, it is possible to change the direction of convergence.

#### Failed Methods
**1. Different methods for crossover** <br>
One other crossover method has been experimented. In that method, two partitions had been chosen to merge the former chromosomes. In other words, the method was cutting a part of chromosome one and put it in the middle of the new chromosome and inherited the others from another chromosome. This method was could not make any significant difference and had lots of overhead on the decoder.<br>
**2. Putting random chromosomes in population** <br>
Another way to randomize the chromosomes more than usual, is to add some random chromosomes in the population. In this way, by choosing these chromosomes as the parents of next generation, the newly added chromosomes will have some random elements as well. Although this method looks like an appropriate solution, it does not have any specific impact on the final answer. 